# 04. Multi-Agent Patterns - LangGraph 완전 정복

## 🎯 학습 목표

**LangGraph의 Multi-Agent 패턴을 Manual Graph 방식으로 구현합니다:**

### 실제 구현 (Manual Graph 방식) ✅
1. 🕸️ **Network** - Agent끼리 직접 통신
2. 🎯 **Supervisor (Manual)** - LLM이 Agent 선택
3. 🏢 **Hierarchical** - 2-level 계층 구조

### 참고용 설명 📚
4. 🤖 **create_react_agent** - Tool calling 필요
5. 🏗️ **LangGraph Supervisor** - 새로운 prebuilt 라이브러리 (2025.2.26 출시)
6. 🐝 **Swarm** - Agent 자율 협업

---

## 💡 왜 Manual Graph?

**Tool Calling의 벽:**
- ❌ 작은 모델 (EXAONE-1.2B, Granite-micro): Tool call 생성 불안정
- ✅ Manual Graph: 모든 모델 호환 + 안정적

**Tool Calling 지원 모델 (OpenAI, Anthropic 등)을 사용한다면:**
- create_react_agent, LangGraph Supervisor 등 prebuilt 사용 가능
- 하지만 로컬 무료 모델로는 Manual Graph가 최선!

---

## 📊 패턴 비교

| 패턴 | 구현 방식 | Tool Calling | 추천 |
|------|----------|--------------|------|
| Network | Manual Graph | 불필요 | ⭐⭐⭐ |
| Supervisor Manual | Manual Graph | 불필요 | ⭐⭐⭐ |
| Hierarchical | Manual Graph | 불필요 | ⭐⭐⭐ |
| create_react_agent | Prebuilt | 필수 | OpenAI만 |
| LangGraph Supervisor | Prebuilt | 필수 | OpenAI만 |
| Swarm | Prebuilt | 필수 | OpenAI만 |


## ⚠️ 환경 설정

**EXAONE-4.0-1.2B 사용 (완전 무료!):**

✅ **로컬 모델 실행**
  - EXAONE-4.0-1.2B (LG AI 한국 모델)
  - Colab GPU 사용
  - 완전 무료, 한도 없음!
  
✅ **Tool Calling 네이티브 지원** ⭐
  - apply_chat_template에 tools 파라미터
  - 변환 불필요!

✅ **작고 빠름**
  - 1.2B 파라미터 (~2.5GB)
  - 빠른 다운로드 & 실행

**준비물:**
- Colab GPU (T4 이상)
- 토큰 불필요 (Public 모델)

**Prebuilt 패키지:**
- `langgraph-supervisor`
- `langgraph-swarm`

In [ ]:
# 패키지 설치
!pip uninstall -y torch torchvision torchaudio
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install -q -U langgraph langchain-core transformers accelerate

# 실제 도구용 패키지
!pip install -q wikipedia  # Wikipedia API

# Prebuilt 패턴용 패키지
!pip install -q -U langgraph-supervisor
!pip install -q -U langgraph-swarm

print("\n✅ 패키지 설치 완료")

# GPU 확인
import torch
if torch.cuda.is_available():
    print(f"   ✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("   ⚠️  CPU 사용 (느림)")

In [ ]:
# EXAONE은 Public 모델 - 토큰 불필요!
print("✅ EXAONE-4.0-1.2B (Public 모델, 토큰 불필요)")
print("   - Manual Graph 방식 사용")
print("   - Tool calling 없이 텍스트 기반으로 작동")


In [ ]:
import os
from typing import TypedDict, Annotated, List, Literal, Any, Optional
import operator
import json
import torch

from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.types import Command
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_core.tools import tool
from pydantic import Field
from IPython.display import Image

from transformers import AutoTokenizer, AutoModelForCausalLM

print("🔄 EXAONE 모델 로딩 중... (~1분)")

model_name = "LGAI-EXAONE/EXAONE-4.0-1.2B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("✅ 모델 로딩 완료!")

# ===== 간단한 LLM Wrapper =====
class SimpleLLM:
    """간단한 LLM 호출 래퍼 (Tool calling 없음)"""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def invoke(self, prompt: str) -> str:
        """텍스트 입력 → 텍스트 출력"""
        messages = [{"role": "user", "content": prompt}]
        
        input_ids = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        
        output = self.model.generate(
            input_ids.to(self.model.device),
            max_new_tokens=512,
            do_sample=True,
            temperature=0.6,
            top_p=0.95,
            pad_token_id=self.tokenizer.eos_token_id
        )
        
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        
        # 마지막 assistant 응답만 추출
        if "assistant" in response:
            response = response.split("assistant")[-1].strip()
        
        return response

# LLM 생성
llm = SimpleLLM(model=model, tokenizer=tokenizer)

print("✅ Simple LLM 초기화 완료")
print(f"   - 모델: EXAONE-4.0-1.2B (LG AI)")
print(f"   - Tool calling: 사용 안 함 (Manual Graph 방식)")
print(f"   - 텍스트 입력 → 텍스트 출력")
print(f"   - GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


## 🔧 실제 도구 및 데이터

**이제부터는 하드코딩이 아닌 실제 도구를 사용합니다!**

### 사용하는 실제 도구들:

1. **Wikipedia API** - 실제 검색
2. **File I/O** - 마크다운/JSON 파일 실제 저장
3. **JSON DB** - 주문/재고 데이터 실제 조회
4. **시뮬레이션** - 환불/재고 등 비즈니스 로직

### 생성되는 실제 결과물:

- `outputs/*.md` - 리서치 보고서, 블로그 포스트
- `outputs/*.json` - 프로젝트 기획서
- `data/orders.json` - 주문 데이터베이스 (자동 생성)

**학생들이 결과를 직접 확인할 수 있습니다!** 🎉

In [ ]:
# 기본 임포트
import os
import json
from datetime import datetime

print("✅ 기본 라이브러리 임포트 완료!")
print("각 패턴별 도구는 해당 섹션에서 정의됩니다.")


---

# 1️⃣ Network Architecture

## 🎯 유즈케이스: Wikipedia 리서치 팀

**시나리오:**
```
사용자: "Rust 프로그래밍 언어에 대해 조사해줘"

→ Research Agent: Wikipedia 실제 검색 🔍
→ LLM 판단: 정보가 충분한가?
  → 충분하면: Writer로 → 파일 저장 💾
  → 부족하면: 추가 검색
```

**핵심:** LLM이 실제로 판단! (하드코딩 ❌)

## 장점

✅ **동적 협업** - LLM이 상황 판단  
✅ **유연함** - 필요시 추가 검색  
✅ **실제 도구** - Wikipedia API + File I/O

## 작동 방식

```python
def research_agent(state):
    result = wikipedia_search.invoke(...)  # 실제 검색!
    
    # LLM에게 판단 시킴
    decision = llm.invoke("정보가 충분한가?")
    
    if "충분" in decision:
        return Command(goto="writer")
    else:
        return Command(goto="research")  # 추가 검색
```

In [ ]:
# ==================== 1. Network: Wikipedia 리서치 ====================

# 📌 Network 전용 도구 (여기!)
@tool
def wikipedia_search(query: str, lang: str = "ko", sentences: int = 3) -> str:
    """Wikipedia에서 실제로 검색합니다 (한국어/영어 지원)
    
    Args:
        query: 검색할 주제
        lang: 언어 (ko/en)
        sentences: 반환할 문장 수
    """
    try:
        import wikipedia
        wikipedia.set_lang(lang)
        result = wikipedia.summary(query, sentences=sentences)
        return f"📚 Wikipedia 검색 결과:\n{result}"
    except wikipedia.exceptions.DisambiguationError as e:
        return f"🔍 여러 결과 발견. 첫 번째 항목 반환:\n{wikipedia.summary(e.options[0], sentences=sentences)}"
    except wikipedia.exceptions.PageError:
        return f"❌ '{query}' 페이지를 찾을 수 없습니다."
    except Exception as e:
        return f"❌ 검색 오류: {str(e)}"

@tool
def save_markdown(filename: str, content: str) -> str:
    """마크다운 파일을 실제로 저장합니다
    
    Args:
        filename: 파일명 (예: report.md)
        content: 마크다운 내용
    """
    os.makedirs("outputs", exist_ok=True)
    filepath = f"outputs/{filename}"
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)
    return f"✅ {filename} 저장 완료! 📄\n경로: {filepath}"

# ===== LLM 응답 클리닝 함수 =====
def clean_llm_response(text: str) -> str:
    """LLM 응답에서 실제 답변만 추출 (EXAONE 특성 고려)"""
    import re
    
    # <think> 태그 제거
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    
    # 줄바꿈 기준으로 분리
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    # 마지막 줄이 보통 실제 답변 (EXAONE 특성)
    if lines:
        # .md나 .json으로 끝나는 줄 찾기
        for line in reversed(lines):
            if line.endswith('.md') or line.endswith('.json') or (len(line) < 50 and not '?' in line):
                return line
        # 없으면 마지막 줄
        return lines[-1]
    
    return text.strip()

# Research Agent (양방향!)
def research_agent_network(state: MessagesState) -> Command[Literal["writer", "research"]]:
    """Wikipedia 검색 → LLM이 다음 agent 결정"""
    print("\n🔍 [Research] Wikipedia 검색 중...")
    
    messages = state["messages"]
    user_query = messages[0].content
    
    # 이미 검색한 횟수 확인
    research_count = sum(1 for m in messages if hasattr(m, 'name') and m.name == "research")
    print(f"   (검색 횟수: {research_count + 1})")
    
    # ✅ LLM이 주제 추출!
    extraction_prompt = f"""다음 사용자 요청에서 Wikipedia에서 검색할 주제만 추출하세요.
주제만 간결하게 1-3단어로 답변하세요. 다른 설명 없이 주제만!

사용자 요청: {user_query}

주제:"""
    
    topic_raw = llm.invoke(extraction_prompt)
    topic = clean_llm_response(topic_raw)
    print(f"   📝 추출된 주제: {topic}")
    
    # Wikipedia 검색
    result = wikipedia_search.invoke({"query": topic, "lang": "ko", "sentences": 3})
    
    # ✅ LLM이 다음 agent 결정! (writer 또는 research 재실행)
    routing_prompt = f"""검색 결과를 분석하고 다음 행동을 결정하세요.

사용자 요청: {user_query}
검색 결과: {result}
검색 횟수: {research_count + 1}

다음 agent를 선택하세요:
- writer: 충분한 정보가 있음 → 보고서 작성
- research: 더 많은 검색 필요 (최대 2회까지만)

한 단어로만 답변하세요 (writer 또는 research):"""
    
    decision_raw = llm.invoke(routing_prompt)
    decision = clean_llm_response(decision_raw).lower()
    
    # 파싱 (기본값: writer)
    if "research" in decision and research_count < 2:
        next_agent = "research"
        print("   🔄 더 필요 → Research 재실행")
    else:
        next_agent = "writer"
        print("   ✅ 충분함 → Writer로 전달")
    
    new_message = HumanMessage(content=result, name="research")
    
    # ✅ 항상 goto 명시!
    return Command(update={"messages": [new_message]}, goto=next_agent)

# Writer Agent (양방향!)
def writer_agent_network(state: MessagesState) -> Command[Literal["research", "writer", "__end__"]]:
    """검색 결과로 보고서 작성 → LLM이 다음 agent 결정"""
    print("\n✍️ [Writer] 보고서 작성 중...")
    
    messages = state["messages"]
    user_query = messages[0].content
    research_results = [m.content for m in messages if hasattr(m, 'name') and m.name == "research"]
    
    writer_count = sum(1 for m in messages if hasattr(m, 'name') and m.name == "writer")
    print(f"   (작성 횟수: {writer_count + 1})")
    
    # LLM이 보고서 작성
    writer_prompt = f"""다음 자료를 바탕으로 마크다운 형식의 간단한 보고서를 작성하세요.

원래 요청: {user_query}
조사 자료:
{''.join(research_results)}

보고서 형식:
# 제목
## 개요
내용...

## 결론
내용...

보고서만 작성하세요 (다른 설명 없이):"""
    
    report = llm.invoke(writer_prompt)
    
    # ✅ LLM이 다음 agent 결정!
    routing_prompt = f"""보고서를 검토하고 다음 행동을 결정하세요.

원래 요청: {user_query}
보고서 초안:
{report[:300]}...

다음 agent를 선택하세요:
- __end__: 보고서 완성됨 → 저장하고 종료 (기본값)
- research: 중요한 정보가 부족함 → 추가 조사
- writer: 내용 수정 필요 → 다시 작성 (최대 1회)

한 단어로만 답변하세요 (__end__ / research / writer):"""
    
    decision_raw = llm.invoke(routing_prompt)
    decision = clean_llm_response(decision_raw).lower()
    
    # 파싱
    if "research" in decision and writer_count < 1:
        next_agent = "research"
        print("   🔄 정보 부족 → Research로 돌아감")
        new_message = AIMessage(content=f"[Writer 피드백] 더 많은 정보 필요", name="writer")
        return Command(update={"messages": [new_message]}, goto=next_agent)
    elif "writer" in decision and writer_count < 1:
        next_agent = "writer"
        print("   🔄 내용 수정 → Writer 재실행")
        new_message = AIMessage(content=report, name="writer")
        return Command(update={"messages": [new_message]}, goto=next_agent)
    else:
        # 완성 → 저장하고 종료
        next_agent = "__end__"
        print("   ✅ 완성 → 저장하고 종료")
        
        # ✅ LLM이 파일명 생성!
        filename_prompt = f"""다음 주제에 적합한 파일명을 생성하세요.
조건: 영어, 소문자, 언더스코어 사용, .md 확장자
예시: quantum_computing_report.md

주제: {user_query}

파일명만 답변하세요 (다른 설명 없이):"""
        
        filename_raw = llm.invoke(filename_prompt)
        filename = clean_llm_response(filename_raw)
        
        # .md 확장자 확인
        if not filename.endswith('.md'):
            filename += '.md'
        
        # 특수문자 제거 (안전하게)
        import re
        filename = re.sub(r'[^a-zA-Z0-9_.]', '_', filename)
        
        print(f"   📝 생성된 파일명: {filename}")
        
        # 파일 저장
        save_result = save_markdown.invoke({"filename": filename, "content": report})
        print(f"   {save_result}")
        
        final_message = AIMessage(content=f"📄 보고서 완성!\n\n{report[:200]}...\n\n{save_result}")
        return Command(update={"messages": [final_message]}, goto=next_agent)

# Graph 구성
network_graph = StateGraph(MessagesState)
network_graph.add_node("research", research_agent_network)
network_graph.add_node("writer", writer_agent_network)

# ✅ START만 연결! Command의 goto가 양방향 처리
network_graph.add_edge(START, "research")

network_app = network_graph.compile()

# 시각화
display(Image(network_app.get_graph().draw_mermaid_png()))

print("\n✅ Network Pattern 준비 완료!")
print("   - Research ↔ Writer (양방향!)")
print("   - LLM 응답 클리닝 추가 (<think> 태그 처리)")
print("   - 파일명 안전하게 생성")


## 📝 실습 미션

### 미션 A: Rust 언어 조사 ⭐⭐
```
"Rust 프로그래밍 언어에 대해 조사해줘"
```
→ `rust_프로그래밍_언어_report.md` 생성 확인

### 미션 B: AI 기술 조사 ⭐⭐⭐
```
"GPT-4에 대해 조사해줘"
```
→ Wikipedia 검색 결과 확인  
→ LLM 판단 메시지 확인

In [ ]:
# Network 실행

print("="*70)
print("🕸️ Network: Wikipedia 리서치")
print("="*70)

result = network_app.invoke(
    {"messages": [HumanMessage(content="Rust 프로그래밍 언어에 대해 조사해줘")]},
    config={"recursion_limit": 50}
)

print("\n✅ 완료!")
print("\n📊 결과:")
print(result['messages'][-1].content)

# 파일 확인
import os
if os.path.exists("outputs"):
    print("\n📁 생성된 파일:")
    for f in os.listdir("outputs"):
        if "report" in f:
            print(f"   ✅ {f}")

---

# 2️⃣ Supervisor - 3가지 구현 방법

## 🎯 유즈케이스: E-커머스 고객 지원

**시나리오:**
```
사용자: "주문 12345 어디까지 왔어?"
→ Supervisor (LLM): 주문 관련 → order_agent 선택
→ Order Agent: data/orders.json 조회
→ 결과: "배송중 (서울 허브)"
```

**핵심:** LLM이 적절한 agent 선택!

## 3가지 방법

| 방법 | 레벨 | 특징 | 사용률 |
|------|------|------|--------|
| 1. Graph 직접 | Low | 완전 제어 | 5% |
| 2. create_react_agent | Mid | Tool 래핑 | **80%** ⭐ |
| 3. create_supervisor | High | 한 줄 | 15% |

**같은 기능, 다른 구현!**

In [ ]:
# ==================== 2. Supervisor 전용 도구 ====================

# 데이터 초기화 (한번만!)
import os
import json
from datetime import datetime

os.makedirs("data", exist_ok=True)

# orders.json 초기화
if not os.path.exists("data/orders.json"):
    orders = {
        "12345": {"status": "배송중", "location": "서울 허브", "expected": "2025-10-17", "product": "노트북"},
        "67890": {"status": "배송완료", "location": "고객 수령", "expected": "2025-10-16", "product": "마우스"},
        "11111": {"status": "준비중", "location": "창고", "expected": "2025-10-18", "product": "키보드"}
    }
    with open("data/orders.json", 'w', encoding='utf-8') as f:
        json.dump(orders, f, ensure_ascii=False, indent=2)
    print("✅ data/orders.json 초기화 완료")

# ===== LLM 응답 클리닝 =====
def clean_llm_response(text: str) -> str:
    """LLM 응답에서 실제 답변만 추출"""
    import re
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    if lines:
        return lines[-1]
    return text.strip()

# 📌 Supervisor 도구 (여기!)
@tool
def search_order(order_id: str) -> str:
    """주문 정보를 조회합니다
    
    Args:
        order_id: 주문 번호
    """
    try:
        with open("data/orders.json", 'r', encoding='utf-8') as f:
            orders = json.load(f)
        
        if order_id in orders:
            order = orders[order_id]
            return f"📦 주문 {order_id}\n상품: {order['product']}\n상태: {order['status']}\n위치: {order['location']}\n배송 예정: {order['expected']}"
        else:
            return f"❌ 주문 {order_id}를 찾을 수 없습니다."
    except Exception as e:
        return f"❌ 조회 오류: {str(e)}"

@tool
def process_refund(order_id: str, reason: str) -> str:
    """주문 환불을 처리합니다
    
    Args:
        order_id: 주문 번호
        reason: 환불 사유
    """
    try:
        with open("data/orders.json", 'r', encoding='utf-8') as f:
            orders = json.load(f)
        
        if order_id not in orders:
            return f"❌ 주문 {order_id}를 찾을 수 없습니다."
        
        # 환불 처리
        orders[order_id]['status'] = '환불완료'
        orders[order_id]['refund_reason'] = reason
        orders[order_id]['refund_date'] = datetime.now().strftime("%Y-%m-%d")
        
        with open("data/orders.json", 'w', encoding='utf-8') as f:
            json.dump(orders, f, ensure_ascii=False, indent=2)
        
        return f"✅ 주문 {order_id} 환불 완료!\n상품: {orders[order_id]['product']}\n사유: {reason}\n처리일: {orders[order_id]['refund_date']}"
    except Exception as e:
        return f"❌ 환불 오류: {str(e)}"

@tool
def check_inventory(product: str) -> str:
    """상품 재고를 확인합니다
    
    Args:
        product: 상품명
    """
    inventory = {
        "노트북": {"stock": 15, "price": 1500000, "category": "전자제품"},
        "마우스": {"stock": 50, "price": 30000, "category": "주변기기"},
        "키보드": {"stock": 0, "price": 80000, "category": "주변기기"},
        "모니터": {"stock": 8, "price": 400000, "category": "전자제품"}
    }
    
    if product in inventory:
        item = inventory[product]
        if item['stock'] > 0:
            return f"✅ {product}\n재고: {item['stock']}개\n가격: {item['price']:,}원\n분류: {item['category']}"
        else:
            return f"❌ {product}: 품절 (가격: {item['price']:,}원)"
    else:
        return f"❌ {product}: 상품 정보 없음"

print("✅ Supervisor 도구 준비 완료:")
print("   - search_order: 주문 조회")
print("   - process_refund: 환불 처리")
print("   - check_inventory: 재고 확인")


---

## 방법 1: Graph 직접 (Command)

**Low-level 완전 제어!**

In [ ]:
# ==================== Supervisor (Graph) ====================

# Worker Agents (LLM이 파라미터 추출!)
def order_agent_sup(state: MessagesState) -> Command[Literal["supervisor", "__end__"]]:
    """주문 조회"""
    print("\n📦 [Order] 조회 중...")
    
    user_msg = state["messages"][0]
    
    # ✅ LLM이 주문번호 추출!
    extraction_prompt = f"""다음 고객 메시지에서 주문 번호를 추출하세요.
숫자만 반환하세요 (5자리). 다른 설명 없이 숫자만!

고객: {user_msg}

주문번호:"""
    
    order_id_raw = llm.invoke(extraction_prompt)
    order_id = clean_llm_response(order_id_raw).strip()
    print(f"   📝 추출된 주문번호: {order_id}")
    
    result = search_order.invoke({"order_id": order_id})
    print(f"   ✅ 완료")
    
    # ✅ Supervisor로 돌아감!
    return Command(goto="supervisor", update={"messages": [AIMessage(content=result)]})

def refund_agent_sup(state: MessagesState) -> Command[Literal["supervisor", "__end__"]]:
    """환불 처리"""
    print("\n💰 [Refund] 처리 중...")
    
    user_msg = state["messages"][0]
    
    # ✅ LLM이 주문번호 + 사유 추출!
    extraction_prompt = f"""다음 고객 메시지에서 정보를 추출하세요.

고객: {user_msg}

JSON 형식으로만 답변:
{{"order_id": "숫자", "reason": "환불 사유"}}"""
    
    extracted_raw = llm.invoke(extraction_prompt)
    extracted = clean_llm_response(extracted_raw)
    
    # 간단한 파싱
    import re
    order_match = re.search(r'"order_id":\s*"([^"]+)"', extracted)
    reason_match = re.search(r'"reason":\s*"([^"]+)"', extracted)
    
    order_id = order_match.group(1) if order_match else "12345"
    reason = reason_match.group(1) if reason_match else "고객 요청"
    
    print(f"   📝 추출: 주문={order_id}, 사유={reason}")
    
    result = process_refund.invoke({"order_id": order_id, "reason": reason})
    print(f"   ✅ 완료")
    
    # ✅ Supervisor로 돌아감!
    return Command(goto="supervisor", update={"messages": [AIMessage(content=result)]})

def inventory_agent_sup(state: MessagesState) -> Command[Literal["supervisor", "__end__"]]:
    """재고 확인"""
    print("\n📊 [Inventory] 확인 중...")
    
    user_msg = state["messages"][0]
    
    # ✅ LLM이 상품명 추출!
    extraction_prompt = f"""다음 고객 메시지에서 상품명을 추출하세요.
상품명만 한글로 반환하세요 (노트북/마우스/키보드/모니터 중 하나). 다른 설명 없이!

고객: {user_msg}

상품명:"""
    
    product_raw = llm.invoke(extraction_prompt)
    product = clean_llm_response(product_raw).strip()
    print(f"   📝 추출된 상품: {product}")
    
    result = check_inventory.invoke({"product": product})
    print(f"   ✅ 완료")
    
    # ✅ Supervisor로 돌아감!
    return Command(goto="supervisor", update={"messages": [AIMessage(content=result)]})

# Supervisor (LLM이 라우팅!)
def supervisor_graph(state: MessagesState) -> Command[Literal["order_agent", "refund_agent", "inventory_agent", "__end__"]]:
    """LLM이 agent 선택"""
    print("\n👔 [Supervisor] LLM 판단 중...")
    
    messages = state["messages"]
    
    # 이미 처리됨? (Worker에서 돌아온 경우)
    if len(messages) > 1:
        print("   ✅ Worker 처리 완료 → 종료")
        # ✅ END로 감!
        return Command(goto="__end__")
    
    user_msg = messages[0]
    
    # ✅ LLM에게 라우팅 판단 시킴!
    routing_prompt = f"""고객 지원 라우터. 다음 중 하나만 선택하세요.

agents:
- order_agent: 주문 조회, 배송 위치 확인
- refund_agent: 환불 처리, 취소
- inventory_agent: 재고 확인, 가격 문의

고객: {user_msg}

한 단어로만 답변하세요 (order_agent / refund_agent / inventory_agent):"""
    
    decision_raw = llm.invoke(routing_prompt)
    decision_text = decision_raw if hasattr(decision_raw, 'content') else str(decision_raw)
    decision_clean = clean_llm_response(decision_text)
    
    print(f"   🤖 LLM 결정: {decision_clean}")
    
    # 파싱
    text = decision_clean.lower()
    if "refund" in text or "환불" in text:
        next_agent = "refund_agent"
    elif "inventory" in text or "재고" in text:
        next_agent = "inventory_agent"
    else:
        next_agent = "order_agent"
    
    print(f"   → {next_agent}")
    
    # ✅ Worker로 이동!
    return Command(goto=next_agent)

# Graph
sup_graph = StateGraph(MessagesState)
sup_graph.add_node("supervisor", supervisor_graph)
sup_graph.add_node("order_agent", order_agent_sup)
sup_graph.add_node("refund_agent", refund_agent_sup)
sup_graph.add_node("inventory_agent", inventory_agent_sup)

# ✅ START만 연결! 나머지는 Command의 goto가 처리
sup_graph.add_edge(START, "supervisor")

sup_graph_app = sup_graph.compile()

# 시각화
display(Image(sup_graph_app.get_graph().draw_mermaid_png()))

print("\n✅ Supervisor (Graph) 완성!")
print("   흐름: START → Supervisor → Worker → Supervisor → END")
print("   - Supervisor: LLM이 agent 라우팅")
print("   - Workers: LLM이 파라미터 추출 → Supervisor로 복귀")
print("   - Supervisor: Worker 결과 확인 → END")


In [ ]:
# Supervisor (Graph) 실행

tests = [
    "주문 12345 어디까지 왔어?",
    "주문 67890 환불해줘",
    "노트북 재고 있어?"
]

for user_msg in tests:
    print("\n" + "="*70)
    print(f"📋 테스트: {user_msg}")
    print("="*70)
    
    result = sup_graph_app.invoke(
        {"messages": [HumanMessage(content=user_msg)]},
        config={"recursion_limit": 50}
    )
    
    print(f"\n✅ 결과: {result['messages'][-1].content[:100]}...")

# 2️⃣-B. Supervisor (create_react_agent 방식)

## 📚 참고용: create_react_agent 패턴

`create_react_agent`는 LangGraph의 prebuilt 함수로, ReAct 패턴을 자동으로 구현합니다.

```python
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    llm,  # Tool calling 지원 LLM
    tools=[handle_order, handle_refund, handle_inventory]
)
```

### 왜 Manual Graph를 선택했나요?

**Tool calling 자체는 EXAONE도 지원합니다!** 문제는:

1. **복잡한 래퍼 구현 필요**
   - AIMessage.tool_calls 파싱
   - 이중 JSON 인코딩 처리
   - 에러 핸들링

2. **작은 모델의 불안정성**
   - Tool call 생성률 낮음
   - 형식 오류 빈번

3. **Manual Graph가 더 간단**
   - LLM → 텍스트 → 정규식 → Tool 실행
   - 명확한 흐름
   - 디버깅 쉬움

**→ Cell 14의 Manual Graph 방식 사용!**


# 2️⃣-C. LangGraph Supervisor (Prebuilt)

## 📚 2025.2.26 출시!

```bash
pip install langgraph-supervisor
```

```python
from langgraph_supervisor import create_supervisor

supervisor_app = create_supervisor(
    agents=[order_agent, refund_agent, inventory_agent],
    supervisor_llm=llm
)
```

**→ Manual Graph (Cell 14)가 더 간단하고 안정적!**


---

# 3️⃣ Hierarchical

## 🎯 유즈케이스: 콘텐츠 제작 회사

**진짜 2-Level 구조:**

```
Top Supervisor (전체 제어 + 최종 승인)
  ├─ Research Team
  │   ├─ Wikipedia Agent (검색 실행)
  │   └─ Research Supervisor (재검색 or 승인 판단)
  │
  └─ Content Team
      ├─ Writer Agent (콘텐츠 작성)
      └─ Content Supervisor (재작성 or 승인 판단)
```

## 🔄 작동 흐름

**1단계: Research Team**
```
Top Supervisor → Wikipedia Agent → Research Supervisor
                                    ├─ 재검색 → Wikipedia Agent (루프)
                                    └─ 승인 → Top Supervisor
```

**2단계: Content Team**
```
Top Supervisor → Writer Agent → Content Supervisor
                                ├─ 재작성 → Writer Agent (루프)
                                └─ 승인 → Top Supervisor
```

**3단계: 최종 승인**
```
Top Supervisor → 파일 저장 → END
```

## 💡 핵심 특징

✅ **각 팀에 Supervisor 있음**
- Research Supervisor: 검색 결과 검토
- Content Supervisor: 작성 결과 검토

✅ **팀 내부 피드백 루프**
- 품질 미달 → 재작업
- 만족 → 상위 Supervisor에게 전달

✅ **Top Supervisor의 역할**
- 전체 흐름 제어
- 팀 간 라우팅
- 최종 승인 및 저장

✅ **Command만 사용**
- `add_edge` 없음!
- 모든 라우팅이 `Command(goto=...)`로 동적 처리


In [ ]:
# ==================== 3. Hierarchical ====================

# 📌 Hierarchical 도구
@tool
def wikipedia_search(query: str, lang: str = "ko", sentences: int = 3) -> str:
    """Wikipedia에서 실제로 검색합니다"""
    try:
        import wikipedia
        wikipedia.set_lang(lang)
        result = wikipedia.summary(query, sentences=sentences)
        return f"📚 Wikipedia 검색 결과:\n{result}"
    except wikipedia.exceptions.DisambiguationError as e:
        return f"🔍 여러 결과 발견. 첫 번째 항목:\n{wikipedia.summary(e.options[0], sentences=sentences)}"
    except wikipedia.exceptions.PageError:
        return f"❌ '{query}' 페이지를 찾을 수 없습니다."
    except Exception as e:
        return f"❌ 검색 오류: {str(e)}"

@tool
def save_project(filename: str, title: str, content: str) -> str:
    """프로젝트 데이터를 JSON으로 저장합니다"""
    os.makedirs("outputs", exist_ok=True)
    filepath = f"outputs/{filename}"
    
    data = {
        "title": title,
        "content": content,
        "created": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    return f"✅ {filename} 저장 완료! 📋\n제목: {title}\n경로: {filepath}"

# ===== LLM 응답 클리닝 =====
def clean_llm_response_hier(text: str) -> str:
    """LLM 응답에서 <think> 태그 제거"""
    import re
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    if lines:
        # 짧고 깨끗한 라인 찾기
        clean_lines = [l for l in lines if len(l) < 100 and not any(c in l for c in ['?', '!', ':', '생성', '답변', '주제', '파일명', '제목'])]
        if clean_lines:
            return clean_lines[0]
        return lines[-1]
    return text.strip()

# ===== 진짜 2-Level Hierarchical =====

# ========== Research Team (2개 노드) ==========

def research_wikipedia_agent(state: MessagesState) -> Command[Literal["research_supervisor"]]:
    """Research Team - Wikipedia Agent (검색)"""
    print("\n🔍 [Research > Wikipedia Agent] 검색 중...")
    
    messages = state["messages"]
    user_request = messages[0].content
    
    # LLM이 주제 추출
    prompt = f"""다음 요청에서 Wikipedia 검색할 주제만 추출하세요.
주제만 간결하게 답변하세요.

요청: {user_request}

주제:"""
    
    topic_raw = llm.invoke(prompt)
    topic = clean_llm_response_hier(topic_raw)
    print(f"   📝 주제: {topic}")
    
    # Wikipedia 검색
    result = wikipedia_search.invoke({"query": topic, "lang": "ko", "sentences": 3})
    print("   ✅ 검색 완료 → Research Supervisor로 전달")
    
    return Command(
        goto="research_supervisor",
        update={"messages": [AIMessage(content=f"[Wikipedia] {result}", name="wikipedia")]}
    )

def research_supervisor_node(state: MessagesState) -> Command[Literal["research_wikipedia_agent", "top_supervisor"]]:
    """Research Team - Supervisor (검토 후 재검색 or 승인)"""
    print("\n👔 [Research > Supervisor] 검토 중...")
    
    messages = state["messages"]
    user_request = messages[0].content
    
    # Wikipedia 결과 찾기
    wiki_result = ""
    for m in messages:
        if hasattr(m, 'name') and m.name == "wikipedia":
            wiki_result = m.content
            break
    
    # 재검색 횟수 확인
    research_count = sum(1 for m in messages if hasattr(m, 'name') and m.name == "wikipedia")
    print(f"   📊 검색 횟수: {research_count}")
    
    # LLM이 판단
    prompt = f"""Wikipedia 검색 결과를 검토하세요.

원래 요청: {user_request}
검색 결과: {wiki_result[:300]}...
검색 횟수: {research_count}

다음 중 하나만 선택:
- 재검색: 정보가 부족함 (최대 2회까지만)
- 승인: 충분한 정보 → Content Team으로 전달

한 단어로만 답변하세요 (재검색 / 승인):"""
    
    decision_raw = llm.invoke(prompt)
    decision = clean_llm_response_hier(decision_raw).lower()
    
    if "재검색" in decision and research_count < 2:
        print("   ❌ 재검색 필요 → Wikipedia Agent로 복귀")
        return Command(goto="research_wikipedia_agent")
    else:
        print("   ✅ 승인 → Top Supervisor로 전달")
        return Command(goto="top_supervisor")

# ========== Content Team (2개 노드) ==========

def content_writer_agent(state: MessagesState) -> Command[Literal["content_supervisor"]]:
    """Content Team - Writer Agent (작성)"""
    print("\n✍️ [Content > Writer Agent] 작성 중...")
    
    messages = state["messages"]
    user_request = messages[0].content
    
    # Research 결과 찾기
    research_results = [m.content for m in messages if hasattr(m, 'name') and m.name == "wikipedia"]
    
    # LLM이 콘텐츠 작성
    prompt = f"""다음 자료를 바탕으로 JSON 형식의 프로젝트 기획서를 작성하세요.

원래 요청: {user_request}
조사 자료:
{''.join(research_results)}

JSON 형식:
{{
  "title": "제목 (한글, 2-5단어)",
  "summary": "요약 (2-3문장)",
  "details": "상세 내용 (3-5문장)"
}}

JSON만 작성하세요:"""
    
    content_raw = llm.invoke(prompt)
    print("   ✅ 작성 완료 → Content Supervisor로 전달")
    
    return Command(
        goto="content_supervisor",
        update={"messages": [AIMessage(content=content_raw, name="writer")]}
    )

def content_supervisor_node(state: MessagesState) -> Command[Literal["content_writer_agent", "top_supervisor"]]:
    """Content Team - Supervisor (검토 후 재작성 or 승인)"""
    print("\n👔 [Content > Supervisor] 검토 중...")
    
    messages = state["messages"]
    user_request = messages[0].content
    
    # Writer 결과 찾기
    writer_result = ""
    for m in messages:
        if hasattr(m, 'name') and m.name == "writer":
            writer_result = m.content
            break
    
    # 재작성 횟수 확인
    writer_count = sum(1 for m in messages if hasattr(m, 'name') and m.name == "writer")
    print(f"   📊 작성 횟수: {writer_count}")
    
    # LLM이 판단
    prompt = f"""작성된 콘텐츠를 검토하세요.

원래 요청: {user_request}
작성 결과: {writer_result[:300]}...
작성 횟수: {writer_count}

다음 중 하나만 선택:
- 재작성: 품질이 부족함 (최대 2회까지만)
- 승인: 만족스러움 → Top Supervisor로 전달

한 단어로만 답변하세요 (재작성 / 승인):"""
    
    decision_raw = llm.invoke(prompt)
    decision = clean_llm_response_hier(decision_raw).lower()
    
    if "재작성" in decision and writer_count < 2:
        print("   ❌ 재작성 필요 → Writer Agent로 복귀")
        return Command(goto="content_writer_agent")
    else:
        print("   ✅ 승인 → Top Supervisor로 전달")
        return Command(goto="top_supervisor")

# ========== Top Supervisor ==========

def top_supervisor_hier(state: MessagesState) -> Command[Literal["research_wikipedia_agent", "content_writer_agent", "__end__"]]:
    """Top Supervisor - 전체 흐름 제어"""
    print("\n👑 [Top Supervisor] 판단 중...")
    
    messages = state["messages"]
    
    # 1. 초기 요청? → Research Team 시작
    if len(messages) == 1:
        print("   → Research Team 시작")
        return Command(goto="research_wikipedia_agent")
    
    # 2. Research 완료? → Content Team 시작
    has_research = any(hasattr(m, 'name') and m.name == "wikipedia" for m in messages)
    has_content = any(hasattr(m, 'name') and m.name == "writer" for m in messages)
    
    if has_research and not has_content:
        print("   → Content Team 시작")
        return Command(goto="content_writer_agent")
    
    # 3. 모두 완료? → 최종 검토 후 파일 저장
    if has_research and has_content:
        print("   → 최종 검토 및 저장")
        
        # Writer 결과 찾기
        writer_result = ""
        for m in messages:
            if hasattr(m, 'name') and m.name == "writer":
                writer_result = m.content
                break
        
        # LLM이 파일명 생성
        user_request = messages[0].content
        prompt = f"""파일명 생성:
조건: 영어, 소문자, 언더스코어, .json
주제: {user_request}
파일명만:"""
        
        filename_raw = llm.invoke(prompt)
        filename = clean_llm_response_hier(filename_raw)
        
        # 파일명 정리
        import re
        filename = re.sub(r'[^a-zA-Z0-9_.]', '_', filename)
        filename = re.sub(r'_+', '_', filename).strip('_')
        if not filename.endswith('.json'):
            filename += '.json'
        
        # LLM이 제목 추출
        prompt = f"""다음 JSON에서 title 값만 추출하세요.

{writer_result[:500]}

title 값만:"""
        
        title_raw = llm.invoke(prompt)
        title = clean_llm_response_hier(title_raw)
        
        print(f"   📝 파일명: {filename}")
        print(f"   📝 제목: {title}")
        
        # 파일 저장
        result = save_project.invoke({
            "filename": filename,
            "title": title,
            "content": writer_result
        })
        print("   ✅ 최종 승인 및 저장 완료 → 종료")
        
        return Command(
            goto="__end__",
            update={"messages": [AIMessage(content=result)]}
        )
    
    # 예상치 못한 상태
    print("   ⚠️  예상치 못한 상태 → 종료")
    return Command(goto="__end__")

# ========== Graph 구성 ==========

hier_graph = StateGraph(MessagesState)

# 노드 추가
hier_graph.add_node("top_supervisor", top_supervisor_hier)
hier_graph.add_node("research_wikipedia_agent", research_wikipedia_agent)
hier_graph.add_node("research_supervisor", research_supervisor_node)
hier_graph.add_node("content_writer_agent", content_writer_agent)
hier_graph.add_node("content_supervisor", content_supervisor_node)

# ✅ START만 연결! 나머지는 모두 Command의 goto가 처리
hier_graph.add_edge(START, "top_supervisor")

# ⚠️ add_edge 없음! Command의 goto만 사용!
# 이유: Command가 동적 라우팅을 처리하므로 정적 엣지 불필요

hier_app = hier_graph.compile()

display(Image(hier_app.get_graph().draw_mermaid_png()))

print("\n✅ Hierarchical (진짜 2-Level) 완성!")
print("=" * 70)
print("구조:")
print("  Top Supervisor (최종 승인)")
print("    ├─ Research Team")
print("    │   ├─ Wikipedia Agent (검색)")
print("    │   └─ Research Supervisor (재검색 or 승인)")
print("    └─ Content Team")
print("        ├─ Writer Agent (작성)")
print("        └─ Content Supervisor (재작성 or 승인)")
print()
print("흐름:")
print("  1. Top Supervisor → Research Wikipedia Agent")
print("  2. Wikipedia Agent → Research Supervisor")
print("  3. Research Supervisor → 재검색 or Top Supervisor")
print("  4. Top Supervisor → Content Writer Agent")
print("  5. Writer Agent → Content Supervisor")
print("  6. Content Supervisor → 재작성 or Top Supervisor")
print("  7. Top Supervisor → 최종 승인 및 저장 → END")
print("=" * 70)


In [ ]:
# Hierarchical 실행

print("="*70)
print("🏢 Hierarchical: 콘텐츠 제작 팀")
print("="*70)

# 테스트: 블록체인 자료 제작
print("\n📋 테스트: 블록체인 기술 소개 자료")
print("-"*70)

result = hier_app.invoke(
    {"messages": [HumanMessage(content="블록체인 기술 소개 자료 만들어줘")]},
    config={"recursion_limit": 50}
)

print(f"\n✅ 최종 결과:")
print(result['messages'][-1].content)

print("\n" + "="*70)
print("💡 Hierarchical (2-Level) 특징:")
print("   - Level 1: Top Supervisor → 팀 그래프 선택")
print("   - Level 2-A: Research Team (Supervisor + Wiki Agent + Analyzer Agent)")
print("   - Level 2-B: Content Team (Supervisor + Writer Agent + Publisher Agent)")
print("   - 팀 단위 독립성과 계층적 협업 구조!")
print("   - Subgraph로 팀 구현 → 재사용 가능")
print("="*70)


# 4️⃣ Swarm

## 📚 참고용: Swarm 패턴

```python
from langgraph_swarm import create_swarm

swarm_app = create_swarm([researcher, writer])
```

**Agent 자율 협업 패턴입니다.**

**→ Network 패턴 (Cell 8)과 유사하게 Manual Graph로 구현 가능합니다!**


---

# 📊 전체 비교

## Manual Graph vs Prebuilt

| 항목 | Manual Graph | Prebuilt (create_react_agent, Supervisor 등) |
|------|-------------|---------------------------------------------|
| **Tool Calling** | 불필요 ✅ | 필수 ❌ |
| **LLM 요구사항** | 모든 모델 OK | OpenAI, Anthropic, Gemini 등만 |
| **복잡도** | 명시적 (직접 제어) | 추상화 (자동 처리) |
| **안정성** | 매우 안정적 | Tool call 품질에 의존 |
| **확장성** | 쉬움 (직접 추가) | 제한적 (API 범위 내) |
| **디버깅** | 쉬움 (흐름 명확) | 어려움 (블랙박스) |
| **비용** | 무료 (로컬) | 유료 (API) |

## 패턴별 비교

| 패턴 | 유즈케이스 | 구현 방식 | Tool Calling | 추천 |
|------|-----------|----------|--------------|------|
| **Network** | Wikipedia 리서치 | Manual Graph | 불필요 | ⭐⭐⭐ 모든 모델 |
| **Supervisor Manual** | 고객 지원 | Manual Graph | 불필요 | ⭐⭐⭐ 모든 모델 |
| **Hierarchical** | 콘텐츠 제작 | Manual Graph | 불필요 | ⭐⭐⭐ 모든 모델 |
| **create_react_agent** | 동적 tool 선택 | Prebuilt | 필수 | OpenAI/Anthropic만 |
| **LangGraph Supervisor** | 계층적 시스템 | Prebuilt (2025.2) | 필수 | OpenAI/Anthropic만 |
| **Swarm** | Agent 자율 협업 | Prebuilt | 필수 | OpenAI/Anthropic만 |

## 실무 선택 가이드

### 로컬 무료 모델 (EXAONE, Llama, Mistral 등):
✅ **Manual Graph 방식 사용**
- Network, Supervisor Manual, Hierarchical 모두 구현 가능
- 안정적이고 확실한 작동

### 상업용 API (OpenAI, Anthropic):
✅ **Prebuilt 사용 권장**
- create_react_agent
- LangGraph Supervisor (최신!)
- langgraph-swarm

### Hybrid 접근:
✅ **Manual Graph + Prebuilt 조합**
- 핵심 로직: Manual Graph
- Tool calling: create_react_agent (API 모델로)

---

**💡 결론:** 
- 로컬 무료 모델로 충분히 강력한 Multi-Agent 시스템 구축 가능!
- Manual Graph가 더 명시적이고 디버깅하기 쉬움
- Prebuilt는 편리하지만 Tool calling 필수
